In [ ]:
using JNeuron, PyCall

Python script used for timing

In [ ]:
pyeval(

import neuron

neuron.h.load_file('stdlib.hoc')
neuron.h.load_file('import3d.hoc')
neuron.h.load_file('stdrun.hoc')

neuron.h('objref this')

Import = neuron.h.Import3d_Neurolucida3()

Import.input('./data/cell2.asc')

imprt = neuron.h.Import3d_GUI(Import, 0)
imprt.instantiate(neuron.h.this)

d_lambda=0.1
frequency=100

for sec in neuron.h.allsec():
    sec.nseg = int((sec.L / (d_lambda*neuron.h.lambda_f(frequency,sec=sec)) + .9)/ 2 )*2 + 1

neuron.h.define_shape()
    
for sec in neuron.h.allsec():
    sec.insert('hh')
    sec.insert('pas')
        
neuron.h.finitialize()

neuron.h.fcurrent()

neuron.h.init()

neuron.h.tstop=1000.0

neuron.h.run()

start=time.time(); neuron.h.run(); end=time.time(); print end-start
)